# Dynamic topic modelling via Non-negative Matrix Factorization (NMF)

Warning: running the whole notebook requires more than 16Gb RAM. 
However you can run steps 1 and 2 separately

In [ ]:
! pip3 install gensim
! pip3 install razdel
! pip3 install pymorphy2

Import some preprocessing tools

In [2]:
import sys
sys.path.insert(0, '../../../preprocessing')
import preprocessing_tools_updt as preprocessing_tools

In [3]:
import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords;
import nltk;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
import re;
from io import StringIO;
from nltk.corpus import stopwords
import gc

In [4]:
# data_file = "~/Projects/project_news/proj_news_viz/interfax_sorted_2008-02-11_2019-01-19.csv"
data_file = "~/Downloads/topic_news/topici_i_rubrici_go.csv"

# Step 1: Preprocessing

Here we will process input .csv line by line as far as the separator "|" happens to be a part of the message body for some entries

In [ ]:
date  = []
url   = []
host  = []
topic = []
title = []
body  = []
with open('../../../../../../project_news/interfax_sorted_2008-02-11_2019-01-19.csv') as news_file:
    for line in news_file:
        fields = line.split("|")
        date.append(fields[0])
        url.append(fields[1])
        host.append(fields[2])
        topic.append(fields[3])
        title.append(fields[4])
        body.append(" ".join(fields[5:]))
    news_file.close()
data_prep = pd.DataFrame({'date' : date,
                          'url'  : url,
                          'host' : host,
                          'topic': topic,
                          'title': title,
                          'body' : body},
                         columns={'date', 'url', 'host', 'topic', 'title', 'body'})
data_prep.info()

In [5]:
data = pd.read_csv(data_file,
                   sep=',',
                   encoding='utf-8',
                   error_bad_lines=False);
# data.columns = ["date", "url", "host", "topic", "title", "body"]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235915 entries, 0 to 235914
Data columns (total 5 columns):
short_topic     235915 non-null object
short_rubric    235915 non-null object
newtext         235915 non-null object
year_text       235915 non-null object
topic_rubric    235915 non-null object
dtypes: object(5)
memory usage: 9.0+ MB


As we can see, 106 entries are lost due to extra separators. However, I assume, it is not critical.

In [6]:
data.head()

,short_topic,short_rubric,newtext,year_text,topic_rubric
0,Библиотека,Первая мировая,бой сопоцкина друскеник закончиться отступлени...,1914-09 бой сопоцкина друскеник закончиться от...,Библиотека/Первая мировая
1,Библиотека,Первая мировая,министерство народный просвещение вид происход...,1914-09 министерство народный просвещение вид ...,Библиотека/Первая мировая
2,Библиотека,Первая мировая,фотограф-корреспондент daily mirror рассказыва...,1914-09 фотограф-корреспондент daily mirror ра...,Библиотека/Первая мировая
3,Библиотека,Первая мировая,штабс-капитан п н нестер день увидеть район же...,1914-09 штабс-капитан п н нестер день увидеть ...,Библиотека/Первая мировая
4,Библиотека,Первая мировая,лицо приехать варшава люблин передавать туда д...,1914-09 лицо приехать варшава люблин передават...,Библиотека/Первая мировая


In [69]:
from datetime import datetime
data["date"] = data["year_text"].apply(lambda x: datetime.strptime(x.split(" ")[0], '%Y-%M'))

In [71]:
data["date"].head(20)

0    1914-01-01 00:09:00
1    1914-01-01 00:09:00
2    1914-01-01 00:09:00
3    1914-01-01 00:09:00
4    1914-01-01 00:09:00
5    2004-01-01 00:05:00
6    2008-01-01 00:04:00
7    2009-01-01 00:03:00
8    2009-01-01 00:11:00
9    2010-01-01 00:11:00
10   2011-01-01 00:06:00
11   2012-01-01 00:11:00
12   2012-01-01 00:11:00
13   2012-01-01 00:03:00
14   2012-01-01 00:05:00
15   2012-01-01 00:12:00
16   2012-01-01 00:12:00
17   2012-01-01 00:09:00
18   2013-01-01 00:12:00
19   2013-01-01 00:12:00
Name: date, dtype: datetime64[ns]

In [ ]:
del data_prep
del date
del url
del host
del topic
del title
del body
gc.collect()

In [ ]:
data["cleaned_text"] = data["body"].apply(preprocessing_tools.clean_text)
data[["cleaned_text"]].head()

In [ ]:
# stopword_ru = stopwords.words('russian')
stopword_ru = "../../../preprocessing/stopwords.txt"
data["tokenized_text"] = data["cleaned_text"].apply(preprocessing_tools.lemmatization, stopword=stopword_ru)
data[["tokenized_text"]].head()

NMF expects input texts as strings

In [ ]:
tockens_clean = data["tokenized_text"].tolist()
texts_clean = [" ".join(x) for x in tockens_clean]

In [ ]:
del data
gc.collect()

Save preprocessed texts into a file

In [ ]:
with open('./clean.texts.txt', 'wt') as out_file:
    for doc in texts_clean:
        out_file.write(doc + '\n')
    out_file.close()

# Step 2: Topic modelling

Now we will read the preprocessed data

In [ ]:
documents = []
with open('clean.texts.txt', 'rt') as in_file:
    for line in in_file:
        documents.append(line.rstrip())
    in_file.close()

In [ ]:
documents[0]

In [ ]:
max_features = 3000
vectorizer = CountVectorizer(analyzer='word', max_features=max_features);
x_counts = vectorizer.fit_transform(documents);
print(x_counts.shape)

In [ ]:
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);

In [ ]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

Let us decompose the document-term matrix and experiment with the number of topics

200 topics

In [ ]:
num_topics = 200
model = NMF(n_components=num_topics, init='nndsvd');
model.fit(xtfidf_norm)

In [14]:
# https://medium.com/mlreview/topic-modeling-with-scikit-learn-e80d33668730
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [ ]:
topics = get_nmf_topics(model, 20)
topics

While some topics are pretty clear:
Topic #1 - что-то связанное с передачей информации
Topic #2 - происшествия, несчастные случаи
Topic #3 - финансы, валюта
Topic #4 - футбол
Topic #5 - судебные дела
Topic #6 - пожары
Topic #7 - инциденты с авиационной техникой
...

Nevertheless, some topics overlap (like 7 and 90)

In [ ]:
topics.to_csv('../nmf.topics.%d.csv' % num_topics)

400 topics

In [ ]:
num_topics = 400
model = NMF(n_components=num_topics, init='nndsvd');
model.fit(xtfidf_norm)

In [ ]:
topics400 = get_nmf_topics(model, 20)
topics400

Now let's try to extract topics from lenta.ru preprocessed corpus

In [7]:
documents = data["newtext"]
documents[:3]

0    бой сопоцкина друскеник закончиться отступлени...
1    министерство народный просвещение вид происход...
2    фотограф-корреспондент daily mirror рассказыва...
Name: newtext, dtype: object

In [8]:
max_features = 3000
vectorizer = CountVectorizer(analyzer='word', max_features=max_features);
x_counts = vectorizer.fit_transform(documents);
print(x_counts.shape)

(235915, 3000)


In [9]:
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);

In [10]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

In [11]:
num_topics = 400
model = NMF(n_components=num_topics, init='nndsvd');
model.fit(xtfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0,
  max_iter=200, n_components=400, random_state=None, shuffle=False,
  solver='cd', tol=0.0001, verbose=0)

In [77]:
import pickle
filename = './nmf.lentaru.sav'
pickle.dump(model, open(filename, 'wb'))

In [15]:
topics = get_nmf_topics(model, 20)
topics

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 391,Topic # 392,Topic # 393,Topic # 394,Topic # 395,Topic # 396,Topic # 397,Topic # 398,Topic # 399,Topic # 400
0,год,команда,миллиард,украина,зарплата,фильм,коллекция,погибнуть,видео,процент,...,газпром,внимание,смерть,граница,интернет,обувь,флаг,создать,неизвестный,газета
1,2013,сборная,объесть,гривна,работник,прокат,представить,ещё,публиковать,опрос,...,контракт,обратить,скончаться,пункт,доступ,кожа,красный,использовать,похитить,писать
2,2014,национальный,сумма,верховный,труд,лента,линейка,погибший,youtube,респондент,...,газовый,привлечь,умереть,пограничник,ru,сумка,государственный,создание,кража,коммерсантъ
3,2012,чемпионат,резерв,сбу,доход,оскар,войти,ранение,снять,опросить,...,концерн,блогер,причина,пограничный,ресурс,аксессуар,изображение,технология,украсть,ведомость
4,2015,игрок,общий,рад,получать,премьера,украшение,двое,кадр,доля,...,холдинг,проблема,возраст,населить,онлайн,основать,символ,материал,злоумышленник,известие
5,2011,капитан,капитал,яценюк,средний,кинотеатр,выпустить,трое,опубликовать,инфляция,...,правление,заметить,год,гражданин,портал,цвета,цвет,использование,преступление,times
6,2010,выступать,составить,рада,уровень,сценарий,марка,данные,снятой,вырасти,...,газопровод,необычный,родиться,государственный,контент,выполнить,цвета,основа,преступник,редактор
7,2009,пилот,выручка,украинец,рабочий,кино,линия,провинция,запечатлеть,голос,...,алексей,портал,болезнь,контроль,электронный,модный,судно,использоваться,скрыться,ссылка
8,последний,играть,чистый,112,выплата,съёмка,аксессуар,гибель,выложить,исследование,...,штраф,некоторый,сердце,российско,распространение,специализироваться,изобразить,специальный,подозревать,сми
9,2008,партнёр,состояние,арсений,плата,экран,посвятить,четыре,разместить,снизиться,...,решение,особый,рак,таможенный,com,спортивный,белый,разработка,похищение,материал


In [66]:
topics.head(30)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 391,Topic # 392,Topic # 393,Topic # 394,Topic # 395,Topic # 396,Topic # 397,Topic # 398,Topic # 399,Topic # 400
0,год,команда,миллиард,украина,зарплата,фильм,коллекция,погибнуть,видео,процент,...,газпром,внимание,смерть,граница,интернет,обувь,флаг,создать,неизвестный,газета
1,2013,сборная,объесть,гривна,работник,прокат,представить,ещё,публиковать,опрос,...,контракт,обратить,скончаться,пункт,доступ,кожа,красный,использовать,похитить,писать
2,2014,национальный,сумма,верховный,труд,лента,линейка,погибший,youtube,респондент,...,газовый,привлечь,умереть,пограничник,ru,сумка,государственный,создание,кража,коммерсантъ
3,2012,чемпионат,резерв,сбу,доход,оскар,войти,ранение,снять,опросить,...,концерн,блогер,причина,пограничный,ресурс,аксессуар,изображение,технология,украсть,ведомость
4,2015,игрок,общий,рад,получать,премьера,украшение,двое,кадр,доля,...,холдинг,проблема,возраст,населить,онлайн,основать,символ,материал,злоумышленник,известие
5,2011,капитан,капитал,яценюк,средний,кинотеатр,выпустить,трое,опубликовать,инфляция,...,правление,заметить,год,гражданин,портал,цвета,цвет,использование,преступление,times
6,2010,выступать,составить,рада,уровень,сценарий,марка,данные,снятой,вырасти,...,газопровод,необычный,родиться,государственный,контент,выполнить,цвета,основа,преступник,редактор
7,2009,пилот,выручка,украинец,рабочий,кино,линия,провинция,запечатлеть,голос,...,алексей,портал,болезнь,контроль,электронный,модный,судно,использоваться,скрыться,ссылка
8,последний,играть,чистый,112,выплата,съёмка,аксессуар,гибель,выложить,исследование,...,штраф,некоторый,сердце,российско,распространение,специализироваться,изобразить,специальный,подозревать,сми
9,2008,партнёр,состояние,арсений,плата,экран,посвятить,четыре,разместить,снизиться,...,решение,особый,рак,таможенный,com,спортивный,белый,разработка,похищение,материал


In [29]:
res = model.transform(xtfidf_norm)

In [44]:
feature_names = vectorizer.get_feature_names()

In [89]:
print(res.shape)
len(feature_names)
model.components_.shape

(235915, 400)


(400, 3000)

In [98]:
dt = pd.DataFrame(res)
dt.columns = ["Topic_%d" % i for i in range(1, 401)]
dt["date"] = data["date"]
dt.reset_index(inplace=True)
dt.head()

,index,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,...,Topic_392,Topic_393,Topic_394,Topic_395,Topic_396,Topic_397,Topic_398,Topic_399,Topic_400,date
0,0,0.002590,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.000152,0.000000,0.00015,0.000003,0.0,0.000000,0.000046,0.000000,0.0,1914-01-01 00:09:00
1,1,0.010208,0.000000,0.0,0.0,0.0,0.000006,0.0,0.000000,0.000762,...,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000167,0.0,1914-01-01 00:09:00
2,2,0.002994,0.000000,0.0,0.0,0.0,0.000018,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1914-01-01 00:09:00
3,3,0.004738,0.000330,0.0,0.0,0.0,0.000012,0.0,0.011757,0.000000,...,0.000000,0.005701,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1914-01-01 00:09:00
4,4,0.003027,0.000964,0.0,0.0,0.0,0.000013,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.0,0.000674,0.000000,0.001765,0.0,1914-01-01 00:09:00


In [100]:
dt_melt = dt.melt('date', var_name='topic', value_name='text')
print(dt_melt.shape)
dt_melt.head()

(94601915, 3)


,date,topic,text
0,1914-01-01 00:09:00,index,0.0
1,1914-01-01 00:09:00,index,1.0
2,1914-01-01 00:09:00,index,2.0
3,1914-01-01 00:09:00,index,3.0
4,1914-01-01 00:09:00,index,4.0


In [102]:
joined_melt_new = dt_melt[(dt_melt.topic!='index') & (dt_melt.topic!='date')]
print(joined_melt_new.shape)
joined_melt_new.topic.value_counts()

(94366000, 3)


Topic_199    235915
Topic_201    235915
Topic_336    235915
Topic_344    235915
Topic_350    235915
Topic_397    235915
Topic_188    235915
Topic_235    235915
Topic_361    235915
Topic_5      235915
Topic_225    235915
Topic_37     235915
Topic_140    235915
Topic_218    235915
Topic_275    235915
Topic_261    235915
Topic_145    235915
Topic_81     235915
Topic_227    235915
Topic_179    235915
Topic_256    235915
Topic_115    235915
Topic_2      235915
Topic_216    235915
Topic_124    235915
Topic_130    235915
Topic_209    235915
Topic_346    235915
Topic_26     235915
Topic_393    235915
              ...  
Topic_77     235915
Topic_165    235915
Topic_22     235915
Topic_99     235915
Topic_146    235915
Topic_366    235915
Topic_132    235915
Topic_396    235915
Topic_23     235915
Topic_370    235915
Topic_312    235915
Topic_104    235915
Topic_247    235915
Topic_144    235915
Topic_381    235915
Topic_24     235915
Topic_129    235915
Topic_70     235915
Topic_45     235915


In [63]:
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]]) for i in topic.argsort()[:-no_top_words - 1:-1])
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(documents[doc_index])

In [65]:
display_topics(model.components_, res.T, feature_names, documents, 20, 4)

Topic 0:
<generator object display_topics.<locals>.<genexpr> at 0x125d6bb88>
премьер-министр рф дмитрий медведев утвердить состав совет русский язык правительство россия соответствующий распоряжение 1 декабрь разместить сайт правительство совет возглавить вице-премьер ольга голодец заместитель стать министр образование дмитрий ливан помимо орган войти ещё 33 человек число который представитель сми образовательный культурный организация также федеральный региональный чиновник совет русский язык утвердить постановление правительство 5 ноябрь 2013 год задача входить обеспечение государственный поддержка развитие язык также содействие изучение распространение 21 ноябрь президент рф владимир путин выступать российский литературный собрание выразить обеспокоенность общий снижение грамотность пренебрежение правило русский язык это отметить русский язык слишком велик традиция разрушить
европейский вещательный компания евк ретранслировать россия программа телеканал euronews намерить провести де

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Extract dominant topics for each document

In [73]:
doc_topic = pd.DataFrame({'date':data["date"],
                          'topic':res.argmax(axis=1),
                          'body':documents},
                  columns=['date', 'topic', 'body'])

In [78]:
doc_topic.to_csv('./nmf.lentaru.dominant_topics.%d.csv' % num_topics)

In [83]:
doc_topic.head(30)

,date,topic,body
0,1914-01-01 00:09:00,27,бой сопоцкина друскеник закончиться отступлени...
1,1914-01-01 00:09:00,135,министерство народный просвещение вид происход...
2,1914-01-01 00:09:00,128,фотограф-корреспондент daily mirror рассказыва...
3,1914-01-01 00:09:00,234,штабс-капитан п н нестер день увидеть район же...
4,1914-01-01 00:09:00,382,лицо приехать варшава люблин передавать туда д...
5,2004-01-01 00:05:00,399,россия мочь компенсировать эстония ущерб нанес...
6,2008-01-01 00:04:00,28,американский актёр уэсли снайпёс четверг приго...
7,2009-01-01 00:03:00,85,нью-йорк арестовать подозревать убийство радио...
8,2009-01-01 00:11:00,21,власть район столица южный корея запустить про...
9,2010-01-01 00:11:00,30,город портленд штат орегон вечером 26 ноябрь а...


# Joypy

In [80]:
import joypy # !pip install joypy

In [ ]:
doc_topic.melt()

In [106]:
labels = sorted(joined_melt_new['date'].unique())

fig, axes = joypy.joyplot(joined_melt_new, by='topic', column='text', figsize=(20, 10),xlabelsize=20,ylabelsize=20,
                          overlap=0.4, fill=True, linecolor="k", linewidth=2,
                          kind='values', fade=True, xrot=90, x_range=[i for i in range(len(labels))],
                          background='white');

ticks_labels = {i:t for i, t in enumerate(labels)}
axes[-1].set_xticks([k for k, v in ticks_labels.items() if k % 6 == 0])
ticks = axes[-1].get_xticks()
axes[-1].set_xticklabels([ticks_labels.get(i, 'stop') for i in ticks]);

KeyboardInterrupt: 